Package Installs

In [16]:
# %pip install numpy
# %pip install scipy
# %pip install torch
# %pip install torchvision

Imports

In [ ]:
import torch
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torch.utils.data import random_split, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

Load Data

In [18]:
dataset = MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
print(len(dataset))

60000


In [19]:
image_tensor, label = dataset[10]
print(image_tensor.shape, label)

torch.Size([1, 28, 28]) 3


In [20]:
print(image_tensor[:,10:15,10:15])
print(torch.max(image_tensor), torch.min(image_tensor))

tensor([[[0.0000, 0.0000, 0.0000, 0.0941, 0.8196],
         [0.0000, 0.0000, 0.3569, 0.5373, 0.9922],
         [0.1569, 0.8392, 0.9804, 0.9961, 0.9961],
         [0.3176, 0.9686, 0.9961, 0.9961, 0.9961],
         [0.0000, 0.4314, 0.9647, 0.9961, 0.9961]]])
tensor(0.9961) tensor(0.)


Data Preprocessing

In [ ]:
BATCH_SIZE = 32
train_data, test_data = random_split(dataset, [51000, 9000])
print("length of Train Datasets: ", len(train_data))
print("length of Validation Datasets: ", len(test_data))

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle = True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle = True)

length of Train Datasets:  51000
length of Validation Datasets:  9000


Model

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(3*3*64, 256)
        self.fc2 = nn.Linear(256, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.view(-1, 3*3*64)
        x = F.relu(self.fc1(x))
        x = F.droupout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

cnn = CNN()
print(cnn)
it = iter(train_loader)
X_batch, y_batch = next(it)
print(cnn.forward(X_batch).shape)

In [ ]:
def fit(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters())
    error = nn.CrossEntropyLoss()
    EPOCHS = 5
    model.train()
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()

            predicted = torch.max(output.data, 1)[1]
            correct += (predicted == var_y_batch).sum()
            if batch_idx % 50 == 0:
                print('Epoch : {} [{}/{} ({:.0f%})]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader),
                    loss.data[0] , float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))

In [ ]:
fit(cnn, train_loader)

In [ ]:
def evaluate(model):
    correct = 0
    for test_imgs, test_labels in test_loader:
        test_img = Variable(test_imgs).float()
        output = model(test_imgs)
        predicted = torch.max(output, 1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy:{:.3f}%".format(float(correct) / (len(test_loader)*BATCH_SIZE)))

In [ ]:
evaluate(cnn)